## Imports

In [15]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os
import geopandas as gpd

## Read-in

In [16]:
df = pd.read_csv('flushing_queens_map_template_popup_pin - Sheet1.csv')

## Image URL collector

In [17]:
df['image_links'] = ''
links = []
for url in df['story_link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find_all('img')
        counter = 0
        for image in images:
            counter += 1
            if 'jpg' in image['src']:
                links.append(image['src'])
                if counter == len(images):
                    links.append('no image found')
                break
    except:
        links.append('no_image')

df['image_links'] = links

In [23]:
df.at[11,'image_links']=('trd-logo-16x9.jpeg')
df.at[12,'image_links']=('trd-logo-16x9.jpeg')
df.at[13,'image_links']=('trd-logo-16x9.jpeg')
df.at[14,'image_links']=('trd-logo-16x9.jpeg')
df.at[15,'image_links']=('trd-logo-16x9.jpeg')
df.at[16,'image_links']=('trd-logo-16x9.jpeg')
df.at[17,'image_links']=('trd-logo-16x9.jpeg')
df.at[18,'image_links']=('trd-logo-16x9.jpeg')

In [24]:
df.image_links

0     https://s11.therealdeal.com/trd/up/2022/11/Mai...
1     https://s11.therealdeal.com/trd/up/2022/10/Flu...
2     https://s13.therealdeal.com/trd/up/2022/09/NY-...
3     https://s14.therealdeal.com/trd/up/2022/04/mai...
4     https://s11.therealdeal.com/trd/up/2019/01/120...
5     https://s14.therealdeal.com/trd/up/2021/08/Mai...
6     https://s13.therealdeal.com/trd/up/2018/08/120...
7     https://s13.therealdeal.com/trd/up/2021/11/mai...
8     https://s11.therealdeal.com/trd/up/2019/01/120...
9     https://s11.therealdeal.com/trd/up/2022/08/Ryb...
10    https://s12.therealdeal.com/trd/up/2020/01/120...
11                                   trd-logo-16x9.jpeg
12                                   trd-logo-16x9.jpeg
13                                   trd-logo-16x9.jpeg
14                                   trd-logo-16x9.jpeg
15                                   trd-logo-16x9.jpeg
16                                   trd-logo-16x9.jpeg
17                                   trd-logo-16

## Article link formatter

In [34]:
df.columns

Index(['full_address', 'developers', 'description', 'story_link',
       'image_links', 'geo_address', 'loc', 'point', 'lat', 'lon', 'altitude',
       'short_description', 'remaining_desc'],
      dtype='object')

In [35]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0     <a href="https://therealdeal.com/2022/11/16/re...
1     <a href="https://therealdeal.com/2022/10/19/fl...
2     <a href="https://therealdeal.com/2022/09/26/br...
3     <a href="https://therealdeal.com/2019/11/14/an...
4     <a href="https://therealdeal.com/2019/01/07/ki...
5     <a href="https://therealdeal.com/2021/08/26/fl...
6     <a href="https://therealdeal.com/2020/02/21/de...
7     <a href="https://therealdeal.com/2021/11/17/ft...
8     <a href="https://therealdeal.com/2019/01/07/ki...
9     <a href="https://therealdeal.com/2022/08/11/ry...
10    <a href="https://therealdeal.com/2020/01/23/fl...
11    <a href="http://flushingcommons.com/" target="...
12                                                  NaN
13    <a href="https://therealdeal.com/sponsored/mer...
14    <a href="https://therealdeal.com/2016/11/16/ch...
15    <a href="https://www.fandtgroup.com/project/on...
16    <a href="https://www.fandtgroup.com/project/qu...
17    <a href="https://unitedgroupny.com/portfol

## Google Maps API Geolocater Setup

In [25]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [26]:
df['geo_address'] = df['full_address'] + ' Flushing, Queens'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [31]:
df.columns

Index(['full_address', 'developers', 'description', 'story_link',
       'image_links', 'geo_address', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [28]:
df

,full_address,developers,description,story_link,image_links,geo_address,loc,point,lat,lon,altitude
0,Willets Point/126-43 and 126-55 39th Avenue,"Related Companies, Sterling Equities","Plans for 2,500 units of affordable housing, 2...",https://therealdeal.com/2022/11/16/related-ste...,https://s11.therealdeal.com/trd/up/2022/11/Mai...,Willets Point/126-43 and 126-55 39th Avenue Fl...,"(126-19 126th St 55 39th, College Point, NY 11...","(40.7819226, -73.8417915, 0.0)",40.781923,-73.841791,0.0
1,138-28 Northern Boulevard,Hang Dong Zhang,Plans for a 14-story mixed-use condominium wit...,https://therealdeal.com/2022/10/19/flushing-de...,https://s11.therealdeal.com/trd/up/2022/10/Flu...,"138-28 Northern Boulevard Flushing, Queens","(138-28 Northern Blvd, Flushing, NY 11354, USA...","(40.7638441, -73.8271573, 0.0)",40.763844,-73.827157,0.0
2,133-09/133-17 37th Avenue and 36-27/51 College...,FSA Capital,"Plans for a 173,000-square-foot, 17-story mixe...",https://therealdeal.com/2022/09/26/brian-puns-...,https://s13.therealdeal.com/trd/up/2022/09/NY-...,133-09/133-17 37th Avenue and 36-27/51 College...,"(133-9 37th Ave, Flushing, NY 11354, USA, (40....","(40.7605684, -73.8345264, 0.0)",40.760568,-73.834526,0.0
3,131-01 39th Avenue (within the rezoned Special...,Chris Jiashu Xu/United Construction & Developm...,"Plans for a 19-story, 664,000-square-foot mixe...",https://therealdeal.com/2019/11/14/another-meg...,https://s14.therealdeal.com/trd/up/2022/04/mai...,131-01 39th Avenue (within the rezoned Special...,"(131-01 39th Ave, Flushing, NY 11354, USA, (40...","(40.75900499999999, -73.83635, 0.0)",40.759005,-73.836350,0.0
4,46-15 Kissena Boulevard,Kimco Realty,"Plans for a 300,000-square-foot mixed-use proj...",https://therealdeal.com/2019/01/07/kimco-is-br...,https://s11.therealdeal.com/trd/up/2019/01/120...,"46-15 Kissena Boulevard Flushing, Queens","(46-15 Kissena Blvd, Queens, NY 11355, USA, (4...","(40.7505653, -73.8185639, 0.0)",40.750565,-73.818564,0.0
5,131-02 40th Road,Jade Century Properties,Plans for a pair of 20-story buildings with a ...,https://therealdeal.com/2021/08/26/flushing-po...,https://s14.therealdeal.com/trd/up/2021/08/Mai...,"131-02 40th Road Flushing, Queens","(131-02 40th Rd, Flushing, NY 11354, USA, (40....","(40.7553969, -73.8373872, 0.0)",40.755397,-73.837387,0.0
6,133-25 37th Avenue,Gary Tsan,"Plans for a 19-story, 360-room hotel, as well ...",https://therealdeal.com/2020/02/21/developer-g...,https://s13.therealdeal.com/trd/up/2018/08/120...,"133-25 37th Avenue Flushing, Queens","(133-25 37th Ave, Flushing, NY 11354, USA, (40...","(40.7608892, -73.83416679999999, 0.0)",40.760889,-73.834167,0.0
7,37-12 Prince Street,F&T Group and SCG America,"Finished the Tangram Tower complex, a 1.2 mill...",https://therealdeal.com/2021/11/17/ft-sgc-amer...,https://s13.therealdeal.com/trd/up/2021/11/mai...,"37-12 Prince Street Flushing, Queens","(37-12 Prince St, Flushing, NY 11354, USA, (40...","(40.7604178, -73.8331616, 0.0)",40.760418,-73.833162,0.0
8,142-20 Franklin Avenue,Pinnacle Group,Plans to convert a 144-unit rent stabilized bu...,https://therealdeal.com/2019/01/07/kimco-is-br...,https://s11.therealdeal.com/trd/up/2019/01/120...,"142-20 Franklin Avenue Flushing, Queens","(142-20 Franklin Ave, Flushing, NY 11355, USA,...","(40.7564189, -73.8227457, 0.0)",40.756419,-73.822746,0.0
9,33-66 Farrington Street/33-70 Farrington Stree...,Century Development Group,"Completed a 15-story, 310-room, 350,000-square...",https://therealdeal.com/2022/08/11/rybak-centu...,https://s11.therealdeal.com/trd/up/2022/08/Ryb...,33-66 Farrington Street/33-70 Farrington Stree...,"(33-70 Farrington St 134 37, Flushing, NY 1135...","(40.7650476, -73.8322696, 0.0)",40.765048,-73.832270,0.0


## Correction section

In [33]:
# df.at[0,'Net decline in footprint, in sf']=('Almost 200K sf')
# df.at[1,'Address(es) of space company vacated or plans to vacate, in sf']=('690 East Middlefield Road, Mountain View. Synopsys is subletting the entire building to Waymo.')

### Change value to percent value
### df['percent_change'] = pd.Series(['{0:.2f}%'.format(val) for val in df['percent_change']], index = df.index)

## HTML popup formatter

In [34]:
df.columns

Index(['full_address', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [36]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    developers = df['developers'].iloc[i]
    image = df['image_links'].iloc[i]
    description = df['description_link'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Developers: <strong>{}'''.format(developers) + '''</strong><br>
    <strong>{}'''.format(description) + '''</strong><br>
    </html>
    '''
    return html


### HTML reservoir

In [36]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [42]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=12)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"Real Estate Development's Rise in Flushing")

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [43]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [44]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [45]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/flushing_queens_map
